# Spectacle de drones

<img src="images/drone_boat.png"/>

Le but est de faire un joli spectacle nocturne. Dans notre cas, on a des drones avec laser qui dessinent des rectangles par groupe de 4 drones (cf ci-dessous).

On a un ensemble de 100 drones qui forment l'image d'un bateau en 3D. Pour nous il s'agit d'un ensemble de points 3D (centre de gravité de chaque drone) qu'on range dans un tableau Numpy 3 par N.

Initialement 

* l'ensemble des drones est centré en (0,0,0) donc si on veut positionner le bateau en (10, 20, 200) il faut faire une translation d'autant sur l'ensemble des drones.
* l'axe principal du bateau est l'axe de $x$, c. à d. que l'avant du bateau est en x=6 et l'arrière en x=-6.

In [ ]:
import numpy as np
import matplotlib.pylab as plt

np.set_printoptions(precision=3)
                 
boat = np.load('boat.npy')
print(boat[:,:5])

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Plot surfaces
def array_to_plottable(P):
    P_split = np.split(P, 7, axis=1)
    return [P_i.reshape(3, 5, 5) for P_i in P_split]

ax.scatter(xs=[0], ys=[0], zs=[0], s=10, c='r')
for P_i in array_to_plottable(boat):
    ax.plot_surface(P_i[0], P_i[1], P_i[2], alpha=0.7, cmap='cool', edgecolor='k')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_xlim(-7, 7)
ax.set_ylim(-4, 4)
ax.set_zlim(-1, 12)
ax.set_aspect('equal')
plt.show()
#ax = fig.add_subplot(projection='3d')
#ax.scatter(boat[0], boat[1], boat[2], marker='o')
#ax.view_init(elev=10, azim=70, roll=0)


On désire déplacer le bateau dans le ciel en lui faisant faire des figures (les unités utilisées sont le mètre et la seconde). Tous les mouvements sont décomposés seconde par seconde, c. à d. qu'on donne la position de tous les drones à chaque seconde (un autre programme se débrouille pour déplacer les drones à leur position suivante).

### Figure 1

Faire tourner le bateau (donc l'ensemble des drones) à l'horizontal autour du point (0,0,80) à une distance de 20 m. Il fait un tour complet en 5 minutes. 

On fera attention à ce que l'orientation du bateau suive le mouvement, c. à d. que l'avant du bateau soit toujours bien orientée. 


In [ ]:
def rotz(theta: float) -> np.ndarray:
    return np.array([
        [np.cos(theta), -np.sin(theta), 0.0],
        [np.sin(theta),  np.cos(theta), 0.0],
        [0.0,            0.0,           1.0]
    ])
    

def circle(boat: np.ndarray, t: float) -> np.ndarray:
    T = 300.0                 # 5 minutes = 300 s
    omega = 2 * np.pi / T
    theta = omega * t

    # Centre du cercle (rayon 20, altitude 80)
    center = np.array([
        20 * np.cos(theta),
        20 * np.sin(theta),
        80.0
    ]).reshape(3, 1)

    # Orientation tangentielle (avant du bateau dans le sens du mouvement)
    R = rotz(theta + np.pi / 2)

    # Rotation + translation
    return R @ boat + center


In [ ]:
# Code to display
# Do not touch this unless you know what you are doing
import matplotlib.animation
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.transforms import Affine2D
import mpl_toolkits.mplot3d.art3d as art3d
matplotlib.rcParams['animation.embed_limit'] = 256 # 256mb max animation size

boat_start=boat

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 100  
plt.ioff()
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# 5min -> 300 sec;
FRAMEPERSEC = 0.2

def animate(t, frampersec=0.2):
    boat_prime = circle(boat_start, t/FRAMEPERSEC)
    plt.cla()
    p = Circle((0, 0), 20, fill=False)
    ax.add_patch(p)
    art3d.pathpatch_2d_to_3d(p, z=80, zdir="z")
    # Plot surface as several meshes
    for P_i in array_to_plottable(boat_prime):
        ax.plot_surface(P_i[0], P_i[1], P_i[2], alpha=0.7, cmap='cool', edgecolor='k')   
    # plt.plot(boat_prime[0,:], boat_prime[1,:], boat_prime[2,:], '.b')
    ax.set_xlim([-30, 30])
    ax.set_ylim([-30, 30])
    ax.set_zlim([50, 110]) # alternative
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.set_aspect('equal')
    ax.set_title(f"t: {t} sec")


matplotlib.animation.FuncAnimation(fig, animate, frames=int(300*FRAMEPERSEC))

### Figure 2

On désire faire tourner le bateau sur son axe principal (l'axe des x) jusqu'à revenir à la position d'origine. En aviation on appelle cela faire un tonneau (rotation de 360 sur l'axe du roulis). Écrire une fonction `barrel_roll(boat, T, t)` qui retourne la position de chaque drone à l'instant `t` si un tour complet dure `T`seconds.

In [ ]:
def rotx(T:float, t:float) -> np.array: # helper function for x-axis rotation
    theta = 2 * np.pi * t / T
    return np.array([
        [1.0,           0.0,            0.0],
        [0.0, np.cos(theta), -np.sin(theta)],
        [0.0, np.sin(theta),  np.cos(theta)]
    ])
    
def barrel_roll(boat: np.array, T:float, t:float) -> np.array:
    R = rotx(T, t)
    return R @ boat


In [ ]:
# Code to display
# You can change this
Tcomplete = 20


fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Do not touch this unless you know what you are doing
def animate2(t):
    boat_prime = barrel_roll(boat, Tcomplete, t)
    plt.cla()
    for P_i in array_to_plottable(boat_prime):
        ax.plot_surface(P_i[0], P_i[1], P_i[2], alpha=0.7, cmap='cool', edgecolor='k')   
    ax.set_xlim([-10, 10])
    ax.set_ylim([-10, 10])
    ax.set_zlim([-5, 15])
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.set_aspect('equal')
    ax.set_title(f"t: {t} sec")

# One frame per second for 2 complete rolls
matplotlib.animation.FuncAnimation(fig, animate2, frames=2*Tcomplete)

### Figure 3

Combiner les figures 1 et 2 pour que le bateau fasse un tonneau en 10 secondes toutes les 30 secondes.

In [ ]:
def combined_move(boat: np.array, t:float) -> np.array:
    ROLL_PERIOD = 30.0   # toutes les 30 s
    ROLL_TIME = 10.0     # durée du tonneau

    phase = t % ROLL_PERIOD

    # 1) Tonneau éventuel (dans le repère du bateau)
    if phase < ROLL_TIME:
        R_roll = rotx(ROLL_TIME, phase)
        boat_local = R_roll @ boat
    else:
        boat_local = boat

    # 2) Mouvement circulaire dans le ciel
    return circle(boat_local, t)


In [ ]:
# Code to display
# Do not touch this unless you know what you are doing
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

FRAMEPERSEC = 1

def animate3(t):
    boat_prime = combined_move(boat, t/FRAMEPERSEC)
    plt.cla()
    p = Circle((0, 0), 20, fill=False)
    ax.add_patch(p)
    art3d.pathpatch_2d_to_3d(p, z=80, zdir="z")
    for P_i in array_to_plottable(boat_prime):
        ax.plot_surface(P_i[0], P_i[1], P_i[2], alpha=0.7, cmap='cool', edgecolor='k')   
    ax.set_xlim([-30, 30])
    ax.set_ylim([-30, 30])
    ax.set_zlim([50, 110])
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.set_aspect('equal')
    ax.set_title(f"t: {t} sec")

# 5min -> 300 sec
matplotlib.animation.FuncAnimation(fig, animate3, frames=int(300*FRAMEPERSEC))

### Plus ?

Soyez créatif : créer d'autres belles animations de notre bateau.

### More?

Be creative: create some more nice animations of our boat.

In [ ]:
'''Personnal code'''
# Author @Rayan
''' Y-axis rotate'''

def roty(T:float, t:float) -> np.array: # helper function for x-axis rotation
    theta = 2 * np.pi * t / T
    return np.array([
        [ np.cos(theta), 0.0,  np.sin(theta)],
        [ 0.0,           1.0,  0.0          ],
        [-np.sin(theta), 0.0,  np.cos(theta)]
    ])

def front_roll(boat: np.array, T:float, t:float) -> np.array:
    R = roty(T, t)
    return R @ boat


In [ ]:
'''Personnal code'''
# Author @Rayan
''' Y-axis rotate plot'''

# Code to display
# You can change this
Tcomplete = 20


fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Do not touch this unless you know what you are doing
def animate2(t):
    boat_prime = front_roll(boat, Tcomplete, t)
    plt.cla()
    for P_i in array_to_plottable(boat_prime):
        ax.plot_surface(P_i[0], P_i[1], P_i[2], alpha=0.7, cmap='cool', edgecolor='k')   
    ax.set_xlim([-10, 10])
    ax.set_ylim([-10, 10])
    ax.set_zlim([-5, 15])
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.set_aspect('equal')
    ax.set_title(f"t: {t} sec")

# One frame per second for 2 complete rolls
matplotlib.animation.FuncAnimation(fig, animate2, frames=2*Tcomplete)

In [ ]:
def doubleroll_combined_move(boat: np.ndarray, t: float) -> np.ndarray:
    # --- Barrel roll (continu) ---
    T_barrel = 40.0
    R_barrel = rotx(T_barrel, t)

    # --- Front roll (period) ---
    FRONT_PERIOD = 30.0
    FRONT_TIME = 10.0
    phase = t % FRONT_PERIOD

    if phase < FRONT_TIME:
        R_front = roty(FRONT_TIME, phase)
    else:
        R_front = np.eye(3)

    # Combine rotations 
    boat_local = R_front @ R_barrel @ boat

    # Global circular motion
    return circle(boat_local, t)

In [ ]:
# Code to display
# Do not touch this unless you know what you are doing
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

FRAMEPERSEC = 1

def animate4(t):
    boat_prime = doubleroll_combined_move(boat, t/FRAMEPERSEC)
    plt.cla()
    p = Circle((0, 0), 20, fill=False)
    ax.add_patch(p)
    art3d.pathpatch_2d_to_3d(p, z=80, zdir="z")
    for P_i in array_to_plottable(boat_prime):
        ax.plot_surface(P_i[0], P_i[1], P_i[2], alpha=0.7, cmap='cool', edgecolor='k')   
    ax.set_xlim([-30, 30])
    ax.set_ylim([-30, 30])
    ax.set_zlim([50, 110])
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.set_aspect('equal')
    ax.set_title(f"t: {t} sec")

# 5min -> 300 sec
matplotlib.animation.FuncAnimation(fig, animate4, frames=int(300*FRAMEPERSEC))